<span style="font-family: Arial; color: #000814; font-size: 24px; font-weight: bold;">Proyecto Final. Notebook No.2 Creación del Data Warehouse.</span>

<span style="font-family: Arial; color: #0077b6; font-size: 18px; font-weight: bold;">Se importan las librerías necesarias.</span>

In [1]:
import pandas as pd
import numpy as np
import boto3
import psycopg2
import configparser

<span style="font-family: Arial; color: #0077b6; font-size: 18px; font-weight: bold;">Se lee el archivo de configuración. Este archivo no se incluye en el repositorio.</span>

In [2]:
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

<span style="font-family: Arial; color: #0077b6; font-size: 18px; font-weight: bold;">Se crea variable postgrs_driver para que lee las credenciales y para usarse en las funciones de inserción de datos.</span>

In [3]:
RDS_HOST = 'tienda-transactional.cjykcg6gmirx.us-east-2.rds.amazonaws.com'
print(RDS_HOST)

tienda-transactional.cjykcg6gmirx.us-east-2.rds.amazonaws.com


In [4]:
postgres_driver = f"""postgresql://{config.get('TRANSACC', 'DB_USER')}:{config.get('TRANSACC', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('TRANSACC', 'DB_PORT')}/{config.get('TRANSACC', 'DB_NAME')}"""

<span style="font-family: Arial; color: #0077b6; font-size: 18px; font-weight: bold;">Se crea la dimensión de la tabla persona.</span>

In [5]:
sql_query = 'SELECT * FROM persona;'
dimPersonas = pd.read_sql(sql_query, postgres_driver)
dimPersonas.to_csv('dimPersonas.csv', index=False)
dimPersonas.head()

,idpersona,tipo_persona,nombre,tipo_documento,num_documento,direccion,telefono,email
0,518494208,proveedor,Morgan Hahn,DNI,149354134161409,"7620 Brian Lodge Apt. 687 Ortizfurt, WI 83186",233.372.,makayla15@yahoo.com
1,194772995,cliente,Jeremy Sanchez,Pasaporte,177566692218882,USNV Reed FPO AE 29082,662-949-,yrichardson@hotmail.com
2,821172229,cliente,Lauren Everett,NIE,122589503164419,"493 Shepherd Turnpike Suite 415 Harpershire, O...",672-397-,william21@gmail.com
3,754774022,cliente,Dan Hernandez,DNI,182064191703046,"527 Diane Locks Suite 818 Emilyfurt, VA 16746",310.228.,rmoody@yahoo.com
4,313305098,proveedor,Kerry Rodriguez,DNI,130128124657671,"7776 Warren Light West Cassiefort, MS 61996",479.550.,webbthomas@hotmail.com


<span style="font-family: Arial; color: #0077b6; font-size: 18px; font-weight: bold;">Se crea la dimensión de la tabla usuario.</span>

In [6]:
sql_query = 'SELECT * FROM usuario;'
df_usuario = pd.read_sql(sql_query, postgres_driver)
df_usuario = df_usuario.rename(columns={'nombre': 'nombre_usuario', 'estado': 'estado_usuario'})
df_usuario.head()

,idusuario,idrol,nombre_usuario,tipo_documento,num_documento,direccion,telefono,email,clave,estado_usuario
0,8519681,10003,Joseph Simmons,DNI,160270278742016,"798 Flores Inlet South Richard, KS 94909",(976)205,youngryan@hotmail.com,"[b'7', b'3', b'a', b'7', b'8', b'3', b'3', b'd...",0
1,4730883,10005,Amber Harrison,Pasaporte,105054972573702,"7227 Johnson Forest North Jonathonport, NM 59493",(819)887,rachelmills@yahoo.com,"[b'c', b'b', b'b', b'2', b'4', b'b', b'1', b'2...",0
2,1126405,10002,Keith Williams,DNI,129550405193737,"41637 Sawyer Track Apt. 125 Wallacemouth, TX 9...",450-411-,bradleymitchell@yahoo.com,"[b'7', b'b', b'7', b'5', b'f', b'1', b'5', b'8...",0
3,7086086,10001,Keith Moore,DNI,168650743121940,"53973 Smith Plaza New Jeffrey, CA 04929",(724)685,nielsenmichelle@hotmail.com,"[b'c', b'c', b'3', b'a', b'9', b'9', b'8', b'4...",1
4,7884808,10002,Natalie Mercer,NIE,187447326345237,"PSC 7255, Box 9372 APO AA 79137",+1-266-5,ruth56@yahoo.com,"[b'a', b'7', b'c', b'd', b'e', b'1', b'b', b'2...",1


In [7]:
sql_query = 'SELECT * FROM rol;'
df_rol = pd.read_sql(sql_query, postgres_driver)
df_rol = df_rol.rename(columns={'nombre': 'nombre_rol', 'estado': 'estado_rol'})
df_rol.head()

,idrol,nombre_rol,descripcion,estado_rol
0,10001,Administrador,Este rol tiene acceso completo al sistema y pu...,1
1,10002,Moderador,Este rol tiene permisos para moderar contenido...,1
2,10003,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
3,10004,Soporte Técnico,Este rol proporciona soporte técnico y tiene a...,1
4,10005,Invitado,Este rol tiene acceso limitado y solo puede ve...,1


In [8]:
df_usuario = df_usuario.merge(df_rol, how='inner', on='idrol')
dimUsuario = df_usuario.drop(['idrol'], axis=1)
dimUsuario.to_csv('dimUsuario.csv', index=False)
dimUsuario.head()

,idusuario,nombre_usuario,tipo_documento,num_documento,direccion,telefono,email,clave,estado_usuario,nombre_rol,descripcion,estado_rol
0,8519681,Joseph Simmons,DNI,160270278742016,"798 Flores Inlet South Richard, KS 94909",(976)205,youngryan@hotmail.com,"[b'7', b'3', b'a', b'7', b'8', b'3', b'3', b'd...",0,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
1,4390921,Sherry Bennett,Pasaporte,119765538504727,"3090 Paula Crossing East Diane, SD 41225",001-665-,mark61@hotmail.com,"[b'b', b'3', b'e', b'5', b'd', b'5', b'4', b'0...",0,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
2,8620048,Kristy Bush,DNI,173325928159262,"80172 Barrera Ranch Lake Charlesview, AZ 98072",001-631-,norrisamber@hotmail.com,"[b'5', b'7', b'2', b'6', b'4', b'9', b'b', b'6...",0,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
3,3041299,Christine Gonzalez,NIE,184220062969887,0971 Alexandra Summit Apt. 974 Port Brianburgh...,783.268.,herrerajodi@gmail.com,"[b'1', b'0', b'7', b'9', b'3', b'9', b'6', b'8...",1,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
4,4343830,Steven Davies,DNI,170416386449441,"159 Joseph Islands Anthonyville, TN 53921",984-490-,thomasmark@hotmail.com,"[b'6', b'8', b'0', b'7', b'5', b'4', b'8', b'1...",1,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1


<span style="font-family: Arial; color: #0077b6; font-size: 18px; font-weight: bold;">Se crea la dimensión de la tabla articulo.</span>

In [9]:
sql_query = 'SELECT * FROM articulo;'
df_articulo = pd.read_sql(sql_query, postgres_driver)
df_articulo = df_articulo.rename(columns={'nombre': 'nombre_articulo', 'descripcion': 'descripcion_articulo', 'estado': 'estado_articulo'})
df_articulo.head()

,idarticulo,idcategoria,codigo,nombre_articulo,precio_venta,stock,descripcion_articulo,imagen,estado_articulo
0,983040,32684,wJKXOFMx,Cuadro Painting,642.32,53,Artículo Cuadro Painting en venta en tienda,fig_0,1
1,638977,97521,3Z49Bd7a,Reloj Behavior,97.48,59,Artículo Reloj Behavior en venta en tienda,fig_1,1
2,270338,39582,fh5B7Qu4,Suéter Inside,495.60,10,Artículo Suéter Inside en venta en tienda,fig_2,1
3,194560,40879,nRKPTUzu,Maquillaje Serve,470.14,87,Artículo Maquillaje Serve en venta en tienda,fig_3,1
4,923655,75369,k1OVBNti,Teléfono Strategy,153.01,54,Artículo Teléfono Strategy en venta en tienda,fig_4,1


In [10]:
sql_query = 'SELECT * FROM categoria;'
df_categoria = pd.read_sql(sql_query, postgres_driver)
df_categoria = df_categoria.rename(columns={'nombre': 'nombre_categoria', 'descripcion': 'descripcion_categoria', 'estado': 'estado_categoria'})
df_categoria.head()

,idcategoria,nombre_categoria,descripcion_categoria,estado_categoria
0,39582,Ropa y Accesorios,"Incluye prendas de vestir para hombres, mujere...",1
1,82016,Calzado,"Incluye zapatos formales, tenis, botas, sandal...",1
2,51470,Juguetes y Juegos,Una amplia gama de juguetes para ninos de toda...,1
3,75369,Electronicos,"Desde telefonos moviles, computadoras portatil...",1
4,20934,Mascotas,"Ofrece productos para el cuidado de mascotas, ...",1


In [11]:
df_articulo = df_articulo.merge(df_categoria, how='inner', on='idcategoria')
dimArticulo = df_articulo.drop(['idcategoria'], axis=1)
dimArticulo.to_csv('dimArticul.csv', index=False)
dimArticulo.head()

,idarticulo,codigo,nombre_articulo,precio_venta,stock,descripcion_articulo,imagen,estado_articulo,nombre_categoria,descripcion_categoria,estado_categoria
0,983040,wJKXOFMx,Cuadro Painting,642.32,53,Artículo Cuadro Painting en venta en tienda,fig_0,1,Hogar y Decoracion,"Ofrece muebles para el hogar, articulos de dec...",1
1,847887,0ZF78yvj,Mesa Last,707.98,67,Artículo Mesa Last en venta en tienda,fig_6,1,Hogar y Decoracion,"Ofrece muebles para el hogar, articulos de dec...",1
2,499726,MJmmMt0c,Mesa Nearly,804.40,84,Artículo Mesa Nearly en venta en tienda,fig_7,1,Hogar y Decoracion,"Ofrece muebles para el hogar, articulos de dec...",1
3,942113,GoxEvMVg,Mesa Money,155.12,27,Artículo Mesa Money en venta en tienda,fig_14,1,Hogar y Decoracion,"Ofrece muebles para el hogar, articulos de dec...",1
4,340001,rHx1PFyn,Cocina Spend,422.33,86,Artículo Cocina Spend en venta en tienda,fig_15,1,Hogar y Decoracion,"Ofrece muebles para el hogar, articulos de dec...",1


<span style="font-family: Arial; color: #0077b6; font-size: 18px; font-weight: bold;">Se crea la dimensión de la tabla venta.</span>

In [12]:
sql_query = 'SELECT * FROM venta;'
df_venta = pd.read_sql(sql_query, postgres_driver)
df_venta = df_venta.rename(columns={'estado': 'estado_venta'})
df_venta.head()

,idventa,idcliente,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_venta
0,636929,218807615,5192591,Recibo,1548290,77699072,2021-06-20,32.23,30445.21,Pendiente
1,802818,362426775,9231609,Factura,5533699,29886468,2015-09-24,72.82,9944.34,Pendiente
2,925698,199304738,3698332,Recibo,5277702,59021317,2019-06-24,66.60,26724.47,En proceso
3,466950,369537545,8967426,Nota de Débito,7122955,57681925,2015-12-24,84.97,26303.25,Completada
4,981001,201190852,4535468,Factura,5711883,98777097,2018-03-03,40.93,71179.31,Pendiente


In [13]:
sql_query = 'SELECT * FROM persona'
df_cliente = pd.read_sql(sql_query, postgres_driver)
df_cliente = df_cliente[df_cliente['tipo_persona']=='cliente']
df_cliente = df_cliente.rename(columns={'idpersona': 'idcliente', 'nombre': 'nombre_cliente', 'tipo_documento': 'tipo_documento_cliente', 'num_documento': 'num_documento_cliente','direccion': 'direccion_cliente', 'telefono': 'telefono_cliente', 'email': 'email_cliente'})
df_cliente.head()

,idcliente,tipo_persona,nombre_cliente,tipo_documento_cliente,num_documento_cliente,direccion_cliente,telefono_cliente,email_cliente
1,194772995,cliente,Jeremy Sanchez,Pasaporte,177566692218882,USNV Reed FPO AE 29082,662-949-,yrichardson@hotmail.com
2,821172229,cliente,Lauren Everett,NIE,122589503164419,"493 Shepherd Turnpike Suite 415 Harpershire, O...",672-397-,william21@gmail.com
3,754774022,cliente,Dan Hernandez,DNI,182064191703046,"527 Diane Locks Suite 818 Emilyfurt, VA 16746",310.228.,rmoody@yahoo.com
5,435355658,cliente,Christopher Phillips,Pasaporte,165991424086024,"005 Angela Springs Apt. 354 South Angela, ME 5...",277.868.,betty38@yahoo.com
6,664700941,cliente,Ashley Morgan MD,NIE,194934328872971,USS Benson FPO AA 80744,+1-228-7,scott93@gmail.com


In [14]:
df_venta_cliente = df_venta.merge(df_cliente, how='inner', on='idcliente')
df_venta_cliente.head()

,idventa,idcliente,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_venta,tipo_persona,nombre_cliente,tipo_documento_cliente,num_documento_cliente,direccion_cliente,telefono_cliente,email_cliente
0,925698,199304738,3698332,Recibo,5277702,59021317,2019-06-24,66.60,26724.47,En proceso,cliente,Samantha Olsen,DNI,139925835644452,"07010 Tara Mount Tannertown, WA 98985",+1-310-5,luke15@gmail.com
1,981001,201190852,4535468,Factura,5711883,98777097,2018-03-03,40.93,71179.31,Pendiente,cliente,Theresa Gaines,NIE,179154902717881,"6320 Barbara Dam Suite 624 North Manueltown, S...",707-585-,mcox@hotmail.com
2,494960,201190852,3112378,Boleta de Venta,6681938,31137142,2017-12-13,39.04,3634.40,Completada,cliente,Theresa Gaines,NIE,179154902717881,"6320 Barbara Dam Suite 624 North Manueltown, S...",707-585-,mcox@hotmail.com
3,177650,201190852,6406880,Boleta de Venta,8619470,69189073,2021-12-03,0.12,51240.41,Pendiente,cliente,Theresa Gaines,NIE,179154902717881,"6320 Barbara Dam Suite 624 North Manueltown, S...",707-585-,mcox@hotmail.com
4,358413,118805658,4326702,Recibo,4642828,60155916,2016-11-09,16.61,20499.82,Completada,cliente,Brittany Alexander,Tarjeta Residencia,143515372700787,"69618 Ruiz Shoals Nancyshire, VA 04563",+1-277-9,justin42@yahoo.com


In [15]:
sql_query = 'SELECT * FROM usuario;'
df_usuario = pd.read_sql(sql_query, postgres_driver)
df_usuario = df_usuario.rename(columns={'nombre': 'nombre_usuario', 'tipo_documento': 'tipo_documento_usuario', 'num_documento': 'num_documento_usuario','direccion': 'direccion_usuario', 'telefono': 'telefono_usuario', 'email': 'email_usuario', 'estado': 'estado_cliente'})
df_usuario.head()

,idusuario,idrol,nombre_usuario,tipo_documento_usuario,num_documento_usuario,direccion_usuario,telefono_usuario,email_usuario,clave,estado_cliente
0,8519681,10003,Joseph Simmons,DNI,160270278742016,"798 Flores Inlet South Richard, KS 94909",(976)205,youngryan@hotmail.com,"[b'7', b'3', b'a', b'7', b'8', b'3', b'3', b'd...",0
1,4730883,10005,Amber Harrison,Pasaporte,105054972573702,"7227 Johnson Forest North Jonathonport, NM 59493",(819)887,rachelmills@yahoo.com,"[b'c', b'b', b'b', b'2', b'4', b'b', b'1', b'2...",0
2,1126405,10002,Keith Williams,DNI,129550405193737,"41637 Sawyer Track Apt. 125 Wallacemouth, TX 9...",450-411-,bradleymitchell@yahoo.com,"[b'7', b'b', b'7', b'5', b'f', b'1', b'5', b'8...",0
3,7086086,10001,Keith Moore,DNI,168650743121940,"53973 Smith Plaza New Jeffrey, CA 04929",(724)685,nielsenmichelle@hotmail.com,"[b'c', b'c', b'3', b'a', b'9', b'9', b'8', b'4...",1
4,7884808,10002,Natalie Mercer,NIE,187447326345237,"PSC 7255, Box 9372 APO AA 79137",+1-266-5,ruth56@yahoo.com,"[b'a', b'7', b'c', b'd', b'e', b'1', b'b', b'2...",1


In [16]:
df_venta_cliente_usuario = df_venta_cliente.merge(df_usuario, how='inner', on='idusuario')
dimVenta = df_venta_cliente_usuario.drop(['idcliente', 'idusuario', 'idrol'], axis=1)
dimVenta.to_csv('dimVenta.csv', index=False)
dimVenta.head()

,idventa,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_venta,tipo_persona,nombre_cliente,...,telefono_cliente,email_cliente,nombre_usuario,tipo_documento_usuario,num_documento_usuario,direccion_usuario,telefono_usuario,email_usuario,clave,estado_cliente
0,925698,Recibo,5277702,59021317,2019-06-24,66.60,26724.47,En proceso,cliente,Samantha Olsen,...,+1-310-5,luke15@gmail.com,Laura Norman,Pasaporte,137041096498865,"8629 Rita Overpass Suite 903 Travisside, NC 98041",897-364-,brandon45@yahoo.com,"[b'f', b'a', b'2', b'e', b'1', b'd', b'2', b'6...",0
1,981001,Factura,5711883,98777097,2018-03-03,40.93,71179.31,Pendiente,cliente,Theresa Gaines,...,707-585-,mcox@hotmail.com,Danielle Mills,DNI,164406115294418,"380 Stafford Rest Suite 084 Lake Chadshire, VT...",544.230.,george58@yahoo.com,"[b'1', b'b', b'5', b'3', b'5', b'1', b'e', b'2...",0
2,494960,Boleta de Venta,6681938,31137142,2017-12-13,39.04,3634.40,Completada,cliente,Theresa Gaines,...,707-585-,mcox@hotmail.com,Gary Brown,DNI,149856481973744,"8115 Morris Ranch Hartmanfort, PA 40963",956.242.,markslaura@gmail.com,"[b'f', b'a', b'e', b'7', b'a', b'5', b'6', b'e...",0
3,183799,Recibo,3036624,37961174,2017-05-22,45.31,47151.04,Cancelada,cliente,Travis Maxwell,...,+1-452-9,moorekelly@gmail.com,Gary Brown,DNI,149856481973744,"8115 Morris Ranch Hartmanfort, PA 40963",956.242.,markslaura@gmail.com,"[b'f', b'a', b'e', b'7', b'a', b'5', b'6', b'e...",0
4,177650,Boleta de Venta,8619470,69189073,2021-12-03,0.12,51240.41,Pendiente,cliente,Theresa Gaines,...,707-585-,mcox@hotmail.com,Rodney Hendricks,DNI,157594144404203,"76126 Lisa Points Suite 687 Michelleburgh, ME ...",(881)667,luiscontreras@gmail.com,"[b'e', b'a', b'a', b'a', b'4', b'1', b'e', b'b...",1


<span style="font-family: Arial; color: #0077b6; font-size: 18px; font-weight: bold;">Se crea la dimensión de la tabla ingreso.</span>

In [31]:
sql_query = 'SELECT * FROM ingreso;'
df_ingreso = pd.read_sql(sql_query, postgres_driver)
df_ingreso = df_ingreso.rename(columns={'estado': 'estado_ingreso'})
df_ingreso.head()

,idingreso,idproveedor,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_ingreso
0,853260289,199304738,8443356,Ticket de Venta,4339716,None,2017-07-26,22.33,32673.41,Reembolsada
1,972408837,922387107,2391640,Ticket de Venta,7616517,None,2017-08-08,89.64,12483.18,Pendiente
2,702410758,287004077,2236226,Nota de Débito,3225610,None,2015-02-27,98.52,56757.62,En proceso
3,213411846,186325619,8561983,Recibo,1839121,None,2015-12-05,45.68,72976.57,En proceso
4,709617672,572445290,8869860,Ticket de Venta,4472852,None,2021-11-11,64.85,11509.19,Pendiente


In [32]:
sql_query = 'SELECT * FROM persona;'
df_proveedor = pd.read_sql(sql_query, postgres_driver)
df_proveedor = df_proveedor[df_proveedor['tipo_persona']=='proveedor']
df_proveedor = df_proveedor.rename(columns={'idpersona': 'idproveedor', 'nombre': 'nombre_proveedor', 'tipo_documento': 'tipo_documento_proveedor', 'num_documento': 'num_documento_proveedor', 'direccion': 'direccion_proveedor', 'telefono': 'telefono_proveedor', 'email': 'email_proveedor'})
df_proveedor.head()

,idproveedor,tipo_persona,nombre_proveedor,tipo_documento_proveedor,num_documento_proveedor,direccion_proveedor,telefono_proveedor,email_proveedor
0,518494208,proveedor,Morgan Hahn,DNI,149354134161409,"7620 Brian Lodge Apt. 687 Ortizfurt, WI 83186",233.372.,makayla15@yahoo.com
4,313305098,proveedor,Kerry Rodriguez,DNI,130128124657671,"7776 Warren Light West Cassiefort, MS 61996",479.550.,webbthomas@hotmail.com
7,364791821,proveedor,Robert Silva,DNI,113049717794830,"84321 Hicks Harbors Philipland, MI 94346",001-233-,bmaynard@hotmail.com
10,962441239,proveedor,Tyler Velazquez,Pasaporte,192883771021329,"7763 Hunt Locks Gabrielleside, VI 99650",(222)417,porterdavid@hotmail.com
13,209471519,proveedor,Ashley Dillon,Tarjeta Residencia,162420219668502,"9943 Clark Wells New James, MS 55585",79232088,nicholasramos@gmail.com


In [33]:
df_ingreso_proveedor = df_ingreso.merge(df_proveedor, how='inner', on='idproveedor')
df_ingreso_proveedor.head()

,idingreso,idproveedor,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_ingreso,tipo_persona,nombre_proveedor,tipo_documento_proveedor,num_documento_proveedor,direccion_proveedor,telefono_proveedor,email_proveedor
0,702410758,287004077,2236226,Nota de Débito,3225610,None,2015-02-27,98.52,56757.62,En proceso,proveedor,John Miranda,Tarjeta Residencia,190078268003743,"5394 Kim Prairie Apt. 274 Shannonchester, SC 4...",301.842.,jhoward@gmail.com
1,286890350,287004077,2365881,Nota de Débito,4010299,None,2019-02-22,85.03,18017.41,Pendiente,proveedor,John Miranda,Tarjeta Residencia,190078268003743,"5394 Kim Prairie Apt. 274 Shannonchester, SC 4...",301.842.,jhoward@gmail.com
2,709617672,572445290,8869860,Ticket de Venta,4472852,None,2021-11-11,64.85,11509.19,Pendiente,proveedor,Kyle Schultz,Tarjeta Residencia,184170742305324,"8636 Simpson Greens Apt. 941 Port Kathrynbury,...",(350)483,kimberlylee@gmail.com
3,934731725,572445290,4665012,Nota de Débito,8351679,None,2023-10-11,85.41,85979.11,En proceso,proveedor,Kyle Schultz,Tarjeta Residencia,184170742305324,"8636 Simpson Greens Apt. 941 Port Kathrynbury,...",(350)483,kimberlylee@gmail.com
4,937039881,530198493,6406880,Nota de Débito,1087508,None,2019-08-21,68.96,62660.13,En proceso,proveedor,Dr. Lisa Williams MD,DNI,188797209190370,"624 Mark Harbors Suite 655 Garciaburgh, HI 30070",311.714.,tamara98@hotmail.com


In [34]:
sql_query = 'SELECT * FROM usuario'
df_usuario = pd.read_sql(sql_query, postgres_driver)
df_usuario = df_usuario.rename(columns={'nombre': 'nombre_usuario', 'tipo_documento': 'tipo_documento_usuario', 'num_documento': 'num_documento_usuario', 'direccion': 'direccion_usuario', 'telefono': 'telefono_usuario', 'email': 'email_usuario', 'estado': 'estado_usuario'})
df_usuario.head()

,idusuario,idrol,nombre_usuario,tipo_documento_usuario,num_documento_usuario,direccion_usuario,telefono_usuario,email_usuario,clave,estado_usuario
0,8519681,10003,Joseph Simmons,DNI,160270278742016,"798 Flores Inlet South Richard, KS 94909",(976)205,youngryan@hotmail.com,"[b'7', b'3', b'a', b'7', b'8', b'3', b'3', b'd...",0
1,4730883,10005,Amber Harrison,Pasaporte,105054972573702,"7227 Johnson Forest North Jonathonport, NM 59493",(819)887,rachelmills@yahoo.com,"[b'c', b'b', b'b', b'2', b'4', b'b', b'1', b'2...",0
2,1126405,10002,Keith Williams,DNI,129550405193737,"41637 Sawyer Track Apt. 125 Wallacemouth, TX 9...",450-411-,bradleymitchell@yahoo.com,"[b'7', b'b', b'7', b'5', b'f', b'1', b'5', b'8...",0
3,7086086,10001,Keith Moore,DNI,168650743121940,"53973 Smith Plaza New Jeffrey, CA 04929",(724)685,nielsenmichelle@hotmail.com,"[b'c', b'c', b'3', b'a', b'9', b'9', b'8', b'4...",1
4,7884808,10002,Natalie Mercer,NIE,187447326345237,"PSC 7255, Box 9372 APO AA 79137",+1-266-5,ruth56@yahoo.com,"[b'a', b'7', b'c', b'd', b'e', b'1', b'b', b'2...",1


In [35]:
dimIngreso = df_ingreso_proveedor.merge(df_usuario, how='inner', on='idusuario')
dimIngreso.drop(['idproveedor', 'idusuario', 'idrol'], axis=1,  inplace=True)
dimIngreso.to_csv('dimIngreso.csv', index=False)
dimIngreso.head()

,idingreso,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_ingreso,tipo_persona,nombre_proveedor,...,telefono_proveedor,email_proveedor,nombre_usuario,tipo_documento_usuario,num_documento_usuario,direccion_usuario,telefono_usuario,email_usuario,clave,estado_usuario
0,702410758,Nota de Débito,3225610,None,2015-02-27,98.52,56757.62,En proceso,proveedor,John Miranda,...,301.842.,jhoward@gmail.com,Amanda Williams,NIE,155351552018256,"2447 Parker Mountains Smithstad, ID 18404",(522)377,kendrafoster@gmail.com,"[b'c', b'a', b'e', b'a', b'd', b'8', b'2', b'e...",0
1,866696729,Factura,7423482,None,2016-11-08,62.17,88490.35,Cancelada,proveedor,Scott Moreno,...,+1-521-8,lschwartz@hotmail.com,Amanda Williams,NIE,155351552018256,"2447 Parker Mountains Smithstad, ID 18404",(522)377,kendrafoster@gmail.com,"[b'c', b'a', b'e', b'a', b'd', b'8', b'2', b'e...",0
2,286890350,Nota de Débito,4010299,None,2019-02-22,85.03,18017.41,Pendiente,proveedor,John Miranda,...,301.842.,jhoward@gmail.com,Alexis Arellano,Pasaporte,144505594601919,"5738 Duran Knoll Jerryton, MH 28602",001-799-,efreeman@yahoo.com,"[b'6', b'f', b'e', b'd', b'7', b'a', b'e', b'3...",1
3,709617672,Ticket de Venta,4472852,None,2021-11-11,64.85,11509.19,Pendiente,proveedor,Kyle Schultz,...,(350)483,kimberlylee@gmail.com,Joshua Chavez,Pasaporte,136167043997653,"34163 James Motorway Barnestown, MI 51722",864.599.,morrisonjames@yahoo.com,"[b'1', b'5', b'4', b'3', b'8', b'e', b'b', b'a...",1
4,934731725,Nota de Débito,8351679,None,2023-10-11,85.41,85979.11,En proceso,proveedor,Kyle Schultz,...,(350)483,kimberlylee@gmail.com,Dawn Miranda,NIE,123844087394000,"271 Larry Crescent Jefferyfort, ME 84404",211-503-,tracysandoval@hotmail.com,"[b'f', b'b', b'5', b'3', b'3', b'a', b'4', b'7...",1


<span style="font-family: Arial; color: #0077b6; font-size: 18px; font-weight: bold;">Se crea la dimensión de la tabla detalle_venta.</span>

In [36]:
sql_query = 'SELECT * FROM detalle_venta;'
df_detalle_venta = pd.read_sql(sql_query, postgres_driver)
df_detalle_venta.head()

,iddetalle_venta,idventa,idarticulo,cantidad,precio,descuento
0,4063241,840968,752383,11,780.50,720.97
1,8060954,846726,791134,30,627.62,132.39
2,7503898,144219,492432,64,345.43,158.51
3,6848542,487791,931726,18,779.96,110.85
4,5701663,669882,217058,28,468.34,423.72


In [37]:
sql_query = 'SELECT * FROM venta;'
df_venta = pd.read_sql(sql_query, postgres_driver)
df_venta = df_venta.rename(columns={'estado': 'estado_venta'})
df_venta.head()

,idventa,idcliente,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_venta
0,636929,218807615,5192591,Recibo,1548290,77699072,2021-06-20,32.23,30445.21,Pendiente
1,802818,362426775,9231609,Factura,5533699,29886468,2015-09-24,72.82,9944.34,Pendiente
2,925698,199304738,3698332,Recibo,5277702,59021317,2019-06-24,66.60,26724.47,En proceso
3,466950,369537545,8967426,Nota de Débito,7122955,57681925,2015-12-24,84.97,26303.25,Completada
4,981001,201190852,4535468,Factura,5711883,98777097,2018-03-03,40.93,71179.31,Pendiente


In [38]:
df_detalle_venta_venta = df_detalle_venta.merge(df_venta, how='inner', on='idventa')
df_detalle_venta_venta.head()

,iddetalle_venta,idventa,idarticulo,cantidad,precio,descuento,idcliente,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_venta
0,4063241,840968,752383,11,780.50,720.97,254376168,1945183,Boleta de Venta,1174793,11785492,2015-04-28,15.11,80085.27,Pendiente
1,5713220,840968,350249,87,613.01,123.34,254376168,1945183,Boleta de Venta,1174793,11785492,2015-04-28,15.11,80085.27,Pendiente
2,6860239,840968,863182,28,831.51,948.80,254376168,1945183,Boleta de Venta,1174793,11785492,2015-04-28,15.11,80085.27,Pendiente
3,6974303,840968,474768,76,149.90,807.28,254376168,1945183,Boleta de Venta,1174793,11785492,2015-04-28,15.11,80085.27,Pendiente
4,6979485,840968,839942,98,535.33,501.36,254376168,1945183,Boleta de Venta,1174793,11785492,2015-04-28,15.11,80085.27,Pendiente


In [39]:
sql_query = 'SELECT *FROM articulo;'
df_articulo = pd.read_sql(sql_query, postgres_driver)
df_articulo = df_articulo.rename(columns={'estado': 'estado_articulo'})
df_articulo.head()

,idarticulo,idcategoria,codigo,nombre,precio_venta,stock,descripcion,imagen,estado_articulo
0,983040,32684,wJKXOFMx,Cuadro Painting,642.32,53,Artículo Cuadro Painting en venta en tienda,fig_0,1
1,638977,97521,3Z49Bd7a,Reloj Behavior,97.48,59,Artículo Reloj Behavior en venta en tienda,fig_1,1
2,270338,39582,fh5B7Qu4,Suéter Inside,495.60,10,Artículo Suéter Inside en venta en tienda,fig_2,1
3,194560,40879,nRKPTUzu,Maquillaje Serve,470.14,87,Artículo Maquillaje Serve en venta en tienda,fig_3,1
4,923655,75369,k1OVBNti,Teléfono Strategy,153.01,54,Artículo Teléfono Strategy en venta en tienda,fig_4,1


In [40]:
dimDetalleVenta = df_detalle_venta_venta.merge(df_articulo, how='inner', on='idarticulo')
dimDetalleVenta.drop(['idventa', 'idarticulo', 'idcliente', 'idusuario'], axis=1,  inplace=True)
dimDetalleVenta.to_csv('dimDetalleventa.csv', index=False)
dimDetalleVenta.head()

,iddetalle_venta,cantidad,precio,descuento,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_venta,idcategoria,codigo,nombre,precio_venta,stock,descripcion,imagen,estado_articulo
0,4063241,11,780.5,720.97,Boleta de Venta,1174793,11785492,2015-04-28,15.11,80085.27,Pendiente,39582,AlUHoAdG,Suéter Himself,780.5,42,Artículo Suéter Himself en venta en tienda,fig_341,1
1,4818419,12,780.5,464.28,Nota de Crédito,1249982,13357780,2018-09-01,27.04,90554.74,Cancelada,39582,AlUHoAdG,Suéter Himself,780.5,42,Artículo Suéter Himself en venta en tienda,fig_341,1
2,9216433,31,780.5,701.73,Nota de Débito,8190903,64781244,2022-10-11,95.83,2694.43,Pendiente,39582,AlUHoAdG,Suéter Himself,780.5,42,Artículo Suéter Himself en venta en tienda,fig_341,1
3,5892399,2,780.5,752.25,Nota de Crédito,7266067,38946606,2022-07-19,4.99,4558.94,En proceso,39582,AlUHoAdG,Suéter Himself,780.5,42,Artículo Suéter Himself en venta en tienda,fig_341,1
4,6277097,39,780.5,671.87,Recibo,5897278,57441336,2020-11-30,98.70,30701.73,Pendiente,39582,AlUHoAdG,Suéter Himself,780.5,42,Artículo Suéter Himself en venta en tienda,fig_341,1


<span style="font-family: Arial; color: #0077b6; font-size: 18px; font-weight: bold;">Se crea la dimensión de la tabla detalle_ingreso.</span>

In [45]:
sql_query = 'SELECT * FROM detalle_ingreso;'
df_detalle_ingreso = pd.read_sql(sql_query, postgres_driver)
df_detalle_ingreso.head()

,iddetalle_ingreso,idingreso,idarticulo,cantidad,precio
0,631697411,484178224,845598,47,603.44
1,486397957,477319365,890834,65,137.68
2,191725574,107799727,818538,59,906.27
3,930101255,448812886,609917,46,176.47
4,719360009,646560677,671028,91,802.04


In [46]:
sql_query = 'SELECT * FROM ingreso;'
df_ingreso = pd.read_sql(sql_query, postgres_driver)
df_ingreso = df_ingreso.rename(columns={'estado': 'estado_ingreso'})
df_ingreso.head()

,idingreso,idproveedor,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_ingreso
0,853260289,199304738,8443356,Ticket de Venta,4339716,None,2017-07-26,22.33,32673.41,Reembolsada
1,972408837,922387107,2391640,Ticket de Venta,7616517,None,2017-08-08,89.64,12483.18,Pendiente
2,702410758,287004077,2236226,Nota de Débito,3225610,None,2015-02-27,98.52,56757.62,En proceso
3,213411846,186325619,8561983,Recibo,1839121,None,2015-12-05,45.68,72976.57,En proceso
4,709617672,572445290,8869860,Ticket de Venta,4472852,None,2021-11-11,64.85,11509.19,Pendiente


In [47]:
df_detalle_ingreso_ingreso = df_detalle_ingreso.merge(df_ingreso, how='inner', on='idingreso')
df_detalle_ingreso_ingreso.head()

,iddetalle_ingreso,idingreso,idarticulo,cantidad,precio,idproveedor,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_ingreso
0,631697411,484178224,845598,47,603.44,186842913,3411279,Recibo,9771267,None,2016-07-14,51.03,10312.86,Reembolsada
1,486397957,477319365,890834,65,137.68,446675633,3863114,Recibo,7731381,None,2020-06-05,2.57,16174.59,Reembolsada
2,760256816,477319365,859012,67,179.50,446675633,3863114,Recibo,7731381,None,2020-06-05,2.57,16174.59,Reembolsada
3,191725574,107799727,818538,59,906.27,493376847,7964655,Nota de Débito,4785224,None,2015-03-04,97.56,87481.42,Pendiente
4,849576871,107799727,290759,45,725.59,493376847,7964655,Nota de Débito,4785224,None,2015-03-04,97.56,87481.42,Pendiente


In [48]:
sql_query = 'SELECT * FROM articulo;'
df_articulo = pd.read_sql(sql_query, postgres_driver)
df_articulo = df_articulo.rename(columns={'estado': 'estado_articulo', 'nombre': 'nombre_articulo'})
df_articulo.head()

,idarticulo,idcategoria,codigo,nombre_articulo,precio_venta,stock,descripcion,imagen,estado_articulo
0,983040,32684,wJKXOFMx,Cuadro Painting,642.32,53,Artículo Cuadro Painting en venta en tienda,fig_0,1
1,638977,97521,3Z49Bd7a,Reloj Behavior,97.48,59,Artículo Reloj Behavior en venta en tienda,fig_1,1
2,270338,39582,fh5B7Qu4,Suéter Inside,495.60,10,Artículo Suéter Inside en venta en tienda,fig_2,1
3,194560,40879,nRKPTUzu,Maquillaje Serve,470.14,87,Artículo Maquillaje Serve en venta en tienda,fig_3,1
4,923655,75369,k1OVBNti,Teléfono Strategy,153.01,54,Artículo Teléfono Strategy en venta en tienda,fig_4,1


In [49]:
dimDetalleIngreso = df_detalle_ingreso_ingreso.merge(df_articulo, how='inner', on='idarticulo')
dimDetalleIngreso.drop(['idingreso', 'idarticulo', 'idproveedor', 'idusuario'], axis=1,  inplace=True)
dimDetalleIngreso.to_csv('dimDetalleIngreso.csv', index=False)
dimDetalleIngreso.head()

,iddetalle_ingreso,cantidad,precio,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_ingreso,idcategoria,codigo,nombre_articulo,precio_venta,stock,descripcion,imagen,estado_articulo
0,631697411,47,603.44,Recibo,9771267,None,2016-07-14,51.03,10312.86,Reembolsada,32684,V6APEyTM,Cocina Between,603.44,28,Artículo Cocina Between en venta en tienda,fig_866,1
1,258779871,7,603.44,Recibo,3465174,None,2022-06-23,31.91,70178.11,Pendiente,32684,V6APEyTM,Cocina Between,603.44,28,Artículo Cocina Between en venta en tienda,fig_866,1
2,486397957,65,137.68,Recibo,7731381,None,2020-06-05,2.57,16174.59,Reembolsada,75369,HQYUL2s0,Tableta View,137.68,37,Artículo Tableta View en venta en tienda,fig_970,1
3,370383305,64,137.68,Recibo,2953294,None,2017-09-05,95.86,41454.85,Reembolsada,75369,HQYUL2s0,Tableta View,137.68,37,Artículo Tableta View en venta en tienda,fig_970,1
4,760256816,67,179.50,Recibo,7731381,None,2020-06-05,2.57,16174.59,Reembolsada,39582,Ecjh8nRq,Suéter Technology,179.50,97,Artículo Suéter Technology en venta en tienda,fig_402,1


<span style="font-family: Arial; color: #0077b6; font-size: 18px; font-weight: bold;">Se crea la dimensión de la tabla rol.</span>

In [50]:
sql_query = 'SELECT * FROM rol;'
dimRol = pd.read_sql(sql_query, postgres_driver)
dimRol.to_csv('dimRol.csv', index=False)
dimRol.head()

,idrol,nombre,descripcion,estado
0,10001,Administrador,Este rol tiene acceso completo al sistema y pu...,1
1,10002,Moderador,Este rol tiene permisos para moderar contenido...,1
2,10003,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
3,10004,Soporte Técnico,Este rol proporciona soporte técnico y tiene a...,1
4,10005,Invitado,Este rol tiene acceso limitado y solo puede ve...,1


<span style="font-family: Arial; color: #0077b6; font-size: 18px; font-weight: bold;">Se crea la dimensión de la tabla categoria.</span>

In [51]:
sql_query = 'SELECT * FROM categoria;'
dimCategoria = pd.read_sql(sql_query, postgres_driver)
dimCategoria.to_csv('dimCategoria.csv', index=False)
dimCategoria.head()

,idcategoria,nombre,descripcion,estado
0,39582,Ropa y Accesorios,"Incluye prendas de vestir para hombres, mujere...",1
1,82016,Calzado,"Incluye zapatos formales, tenis, botas, sandal...",1
2,51470,Juguetes y Juegos,Una amplia gama de juguetes para ninos de toda...,1
3,75369,Electronicos,"Desde telefonos moviles, computadoras portatil...",1
4,20934,Mascotas,"Ofrece productos para el cuidado de mascotas, ...",1


<span style="font-family: Arial; color: #0077b6; font-size: 18px; font-weight: bold;">Se crea la dimensión de la tabla tiempo.</span>

In [52]:
sql_query = 'SELECT fecha FROM venta;'
dimFecha = pd.read_sql(sql_query, postgres_driver)
dimFecha.head()

,fecha
0,2021-06-20
1,2015-09-24
2,2019-06-24
3,2015-12-24
4,2018-03-03


In [53]:
dimFecha['year'] = pd.DatetimeIndex(dimFecha['fecha']).year
dimFecha['month'] = pd.DatetimeIndex(dimFecha['fecha']).month
dimFecha['quarter'] = pd.DatetimeIndex(dimFecha['fecha']).quarter
dimFecha['day'] = pd.DatetimeIndex(dimFecha['fecha']).day
dimFecha['week'] = pd.DatetimeIndex(dimFecha['fecha']).week
dimFecha['dayofweek'] = pd.DatetimeIndex(dimFecha['fecha']).dayofweek
#dimFechaHora['hour'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).hour
#dimFechaHora['minute'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).minute
dimFecha['is_weekend'] = dimFecha['dayofweek'].apply(lambda x: 1 if x > 5 else 0)
dimFecha.head()

/var/folders/t4/ltm_f1s92xd8rkv3v0nc5ll80000gn/T/ipykernel_27611/1606713047.py:5: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  dimFecha['week'] = pd.DatetimeIndex(dimFecha['fecha']).week


,fecha,year,month,quarter,day,week,dayofweek,is_weekend
0,2021-06-20,2021,6,2,20,24,6,1
1,2015-09-24,2015,9,3,24,39,3,0
2,2019-06-24,2019,6,2,24,26,0,0
3,2015-12-24,2015,12,4,24,52,3,0
4,2018-03-03,2018,3,1,3,9,5,0


In [54]:
dimFecha['id_fecha'] = dimFecha['year'].astype(str) + dimFecha['month'].astype(str)
dimFecha['id_fecha'] = dimFecha['id_fecha'].astype(str) + dimFecha['day'].astype(str)
#dimFecha['id_fecha'] = dimFecha['id_fecha'].astype(str) + dimFechaHora['hour'].astype(str)
#dimFechaHora['id_fecha'] = dimFechaHora['id_fecha'].astype(str) + dimFechaHora['minute'].astype(str)
dimFecha.drop_duplicates(inplace=True)
dimFecha.to_csv('dimFecha.csv', index=False)
dimFecha.head()

,fecha,year,month,quarter,day,week,dayofweek,is_weekend,id_fecha
0,2021-06-20,2021,6,2,20,24,6,1,2021620
1,2015-09-24,2015,9,3,24,39,3,0,2015924
2,2019-06-24,2019,6,2,24,26,0,0,2019624
3,2015-12-24,2015,12,4,24,52,3,0,20151224
4,2018-03-03,2018,3,1,3,9,5,0,201833


#### Tabla de Hechos
<span style="font-family: Arial; color: #0077b6; font-size: 18px; font-weight: bold;">Se crea la tabla de hechos.</span>

In [55]:
sql_query = '''SELECT idventa, fecha, idcliente,
                              idusuario, tipo_comprobante, serie_comprobante, num_comprobante, impuesto, total,
                              estado FROM venta;'''
df_factTable = pd.read_sql(sql_query, postgres_driver)
df_factTable.to_csv('Tabla_de_Hechos.csv', index=False)
df_factTable.head()

,idventa,fecha,idcliente,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,impuesto,total,estado
0,636929,2021-06-20,218807615,5192591,Recibo,1548290,77699072,32.23,30445.21,Pendiente
1,802818,2015-09-24,362426775,9231609,Factura,5533699,29886468,72.82,9944.34,Pendiente
2,925698,2019-06-24,199304738,3698332,Recibo,5277702,59021317,66.60,26724.47,En proceso
3,466950,2015-12-24,369537545,8967426,Nota de Débito,7122955,57681925,84.97,26303.25,Completada
4,981001,2018-03-03,201190852,4535468,Factura,5711883,98777097,40.93,71179.31,Pendiente
